# Prepare nice json competenframeworksfor Flask app

In [11]:
import json
import requests
import pprint

base_url = 'http://141.5.108.59:3000/ims/case/v1p0'

In [12]:
r = requests.get(base_url + '/CFDocuments')

list_of_documents = []

for item in r.json()['CFDocuments']:
   list_of_documents.append(item['identifier'])  

list_of_documents

['1e2537a8-3b92-11ea-800f-0242ac1a0003',
 '05a9d6f8-3b96-11ea-bc94-0242ac1a0003']

## Get JSON for each document

In [14]:
competence_frameworks = []

for item in list_of_documents:

    r = requests.get(base_url + '/CFPackages/' + item)

    competence_framework = []
    for item in r.json()['CFAssociations']:
        if item['associationType'] == 'isChildOf':
            competence_framework.append(
                (item['destinationNodeURI']['identifier'],(item['originNodeURI']['identifier']))
                )

    parents, children = zip(*competence_framework)
    root_nodes = {x for x in parents if x not in children}
    for node in root_nodes:
        print(node)
        r = requests.get('http://141.5.108.59:3000/ims/case/v1p0/CFDocuments/' + node)
        print(r.json())
        str_title = r.json()['creator']
        print(str_title)
        competence_framework.append(('Root', node))
        competence_framework.append(('Title', r.json()['creator']))

    def get_nodes(node):
        d = {}
        try:
            if node == "Root":
                d['identifier'] = r.json()['creator']
            else:
                d['identifier'] = node
            r = requests.get('http://141.5.108.59:3000/ims/case/v1p0/CFItems/' + d['identifier'])

            if r.status_code == 404:
                # this might be a package URI
                r = requests.get('http://141.5.108.59:3000/ims/case/v1p0/CFDocuments/' + d['identifier'])
                d['title'] = r.json()['title']
                d['creator'] = r.json()['creator']
                d['language'] = r.json()['language']
            else:
                d['title'] = r.json()['humanCodingScheme']
                d['educationLevel'] = r.json()['educationLevel']
                d['fullStatement'] = r.json()['fullStatement']

        except:
            pass
        children = get_children(node)
        if children:
            d['nodes'] = [get_nodes(child) for child in children]
        return d

    def get_children(node):
        return [x[1] for x in competence_framework if x[0] == node]

    tree = get_nodes('Root')
    competence_frameworks.append(tree)

pprint.pprint(competence_frameworks)

1e2537a8-3b92-11ea-800f-0242ac1a0003
{'uri': 'http://141.5.108.59:3000/uri/1e2537a8-3b92-11ea-800f-0242ac1a0003', 'identifier': '1e2537a8-3b92-11ea-800f-0242ac1a0003', 'lastChangeDateTime': '2020-01-20T14:35:34+00:00', 'CFPackageURI': {'title': 'Bildungsstandards im Fach Deutsch für den Primarbereich', 'identifier': '1e2537a8-3b92-11ea-800f-0242ac1a0003', 'uri': 'http://141.5.108.59:3000/uri/p1e2537a8-3b92-11ea-800f-0242ac1a0003'}, 'creator': 'Kultusministerkonferenz', 'title': 'Bildungsstandards im Fach Deutsch für den Primarbereich', 'language': 'de'}
Kultusministerkonferenz
05a9d6f8-3b96-11ea-bc94-0242ac1a0003
{'uri': 'http://141.5.108.59:3000/uri/05a9d6f8-3b96-11ea-bc94-0242ac1a0003', 'identifier': '05a9d6f8-3b96-11ea-bc94-0242ac1a0003', 'lastChangeDateTime': '2020-01-20T15:19:09+00:00', 'CFPackageURI': {'title': 'Teilrahmenplan Deutsch', 'identifier': '05a9d6f8-3b96-11ea-bc94-0242ac1a0003', 'uri': 'http://141.5.108.59:3000/uri/p05a9d6f8-3b96-11ea-bc94-0242ac1a0003'}, 'creator': 'M

In [93]:
competence_frameworks = []

for item in list_of_documents:

    r = requests.get(base_url + '/CFPackages/1e2537a8-3b92-11ea-800f-0242ac1a0003')

    competence_framework = []
    for item in r.json()['CFAssociations']:
        if item['associationType'] == 'isChildOf':
            competence_framework.append(
                (item['destinationNodeURI']['identifier'],(item['originNodeURI']['identifier']))
                )

    parents, children = zip(*competence_framework)
    root_nodes = {x for x in parents if x not in children}
    for node in root_nodes:
        competence_framework.append(('Root', node))

    def get_nodes(node):
        d = {}
        d['identifier'] = node
        try:
            r = requests.get('http://141.5.108.59:3000/ims/case/v1p0/CFItems/' + d['identifier'])

            if r.status_code == 404:
                # this might be a package URI
                r = requests.get('http://141.5.108.59:3000/ims/case/v1p0/CFDocuments/' + d['identifier'])
                #d['title'] = r.json()['title']
                #d['creator'] = r.json()['creator']
                #d['language'] = r.json()['language']
            else:
                pass
                #d['title'] = r.json()['humanCodingScheme']
                #d['educationLevel'] = r.json()['educationLevel']
                #d['fullStatement'] = r.json()['fullStatement']
        except:
            pass
        children = get_children(node)
        if children:
            d['nodes'] = [get_nodes(child) for child in children]
        return d

    def get_children(node):
        return [x[1] for x in competence_framework if x[0] == node]

    tree = get_nodes('Root')
    competence_frameworks.append(tree)

pprint.pprint(competence_frameworks)

[{'identifier': 'Root',
  'nodes': [{'identifier': '1e2537a8-3b92-11ea-800f-0242ac1a0003',
             'nodes': [{'identifier': '2d553cfe-3b93-11ea-9ddf-0242ac1a0003',
                        'nodes': [{'identifier': '6cba87aa-3b93-11ea-97b6-0242ac1a0003'},
                                  {'identifier': '038cc8e6-3b94-11ea-9db2-0242ac1a0003'},
                                  {'identifier': '5f959e38-3b94-11ea-a209-0242ac1a0003'},
                                  {'identifier': 'a81a6b98-3b94-11ea-b0b0-0242ac1a0003'},
                                  {'identifier': 'ceeeaf4a-3b94-11ea-989e-0242ac1a0003'}]},
                       {'identifier': '173a2110-69c3-11ea-b850-0242ac1a0003',
                        'nodes': [{'identifier': '24a40096-69c3-11ea-8722-0242ac1a0003',
                                   'nodes': [{'identifier': '30a5a17e-69c3-11ea-ab63-0242ac1a0003'}]}]}]}]},
 {'identifier': 'Root',
  'nodes': [{'identifier': '1e2537a8-3b92-11ea-800f-0242ac1a0003',
            